# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

## CLASSE REFACTORING: boundingbox

Variabili utili per testare il funzionamento


In [2]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


┌ Warning: Package LinearAlgebraicRepresentation does not have BenchmarkTools in its dependencies:
│ - If you have LinearAlgebraicRepresentation checked out for development and have
│   added BenchmarkTools as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LinearAlgebraicRepresentation
└ Loading BenchmarkTools into LinearAlgebraicRepresentation from project dependency, future warnings for LinearAlgebraicRepresentation are suppressed.


In [3]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);    #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);   #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];             #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## Versione iniziale di boundingbox

La funzione boundingbox serve a creare il bounding Box di una cella, cioè la scatola di misura più piccola (area, volume, ipervolume) entro cui sono contenuti tutti i punti.

In [4]:
function boundingbox(vertices::Lar.Points)
   minimum = mapslices(x->min(x...), vertices, dims=2)
   maximum = mapslices(x->max(x...), vertices, dims=2)
   return minimum, maximum
end

boundingbox (generic function with 1 method)

In [5]:
@btime boundingbox(V)   #14,076 μs

  13.451 μs (147 allocations: 5.03 KiB)


([0.0; 0.0;;], [2.0; 1.0;;])

### versione modificata boundingbox

In [6]:
function boundingboxMOD(vertices::Lar.Points)
    firstDim = vertices[1,:]
    secondDim = vertices[2,:]
    if (size(vertices,1)==3)
        thirdDim = vertices[3,:]
         minimum = Threads.@spawn hcat([min(firstDim...), min(secondDim...), min(thirdDim...)])
         maximum = Threads.@spawn hcat([max(firstDim...), max(secondDim...), max(thirdDim...)])
    else
         minimum = Threads.@spawn hcat([min(firstDim...), min(secondDim...)])
         maximum = Threads.@spawn hcat([max(firstDim...), max(secondDim...)])
    end
    return fetch(minimum),fetch(maximum)
 end

boundingboxMOD (generic function with 1 method)

In [7]:
@btime boundingboxMOD(V)   #14,076 μs

  20.286 μs (63 allocations: 2.19 KiB)


([0.0; 0.0;;], [2.0; 1.0;;])